In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_data = pd.read_csv('/Users/seungwooseo/Desktop/Python/AI-IN-MEDICAL-MATERIALS/02-CNN-Convolutional-Neural-Networks/MNIST_Fashion/fashion-mnist_train.csv')
test_data = pd.read_csv('/Users/seungwooseo/Desktop/Python/AI-IN-MEDICAL-MATERIALS/02-CNN-Convolutional-Neural-Networks/MNIST_Fashion/fashion-mnist_test.csv')

In [4]:
transform = transforms.ToTensor()

In [5]:
class FashionData(Dataset):
    
    def __init__(self, data, transform = None):
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
            label.append(i[0])
            image.append(i[1:])
            
        self.labels = np.asarray(label)
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32') 
        
    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    
    def __len__(self):
        return len(self.images)
    

In [6]:
train_set = FashionData(train_data, transform=transform)
test_set = FashionData(test_data, transform=transform)

In [7]:
train_loader = DataLoader(train_set, batch_size=600, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)

### CNN_ReLU_SoftMax

In [8]:
class MultiClassPerceptron(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [9]:
torch.manual_seed(42)
model = MultiClassPerceptron()
model

MultiClassPerceptron(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
import time
start_time = time.time()

epochs = 10
trn_losses = []
tst_losses = []
trn_correct = []
tst_correct =[]

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1
        
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{600*b:6}/60000]  loss: {loss.item():10.8f} \ accuracy: {trn_corr.item()*100/(600*b):7.3f}%')
            
    trn_losses.append(loss.item())
    trn_correct.append(trn_corr.item())
    
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            
            y_val = model(X_test)
            
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()
            
    
    loss = criterion(y_val, y_test)
    tst_losses.append(loss)
    tst_correct.append(tst_corr)
    

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

epoch:  0  batch:   10 [  6000/60000]  loss: 1.37879682 \ accuracy:  32.467%
epoch:  0  batch:   20 [ 12000/60000]  loss: 0.95504701 \ accuracy:  47.158%
epoch:  0  batch:   30 [ 18000/60000]  loss: 0.67971492 \ accuracy:  55.300%
epoch:  0  batch:   40 [ 24000/60000]  loss: 0.61472809 \ accuracy:  60.438%
epoch:  0  batch:   50 [ 30000/60000]  loss: 0.58287525 \ accuracy:  64.010%
epoch:  0  batch:   60 [ 36000/60000]  loss: 0.55089504 \ accuracy:  66.547%
epoch:  0  batch:   70 [ 42000/60000]  loss: 0.55367190 \ accuracy:  68.438%
epoch:  0  batch:   80 [ 48000/60000]  loss: 0.54955930 \ accuracy:  70.037%
epoch:  0  batch:   90 [ 54000/60000]  loss: 0.45770544 \ accuracy:  71.417%
epoch:  0  batch:  100 [ 60000/60000]  loss: 0.44206715 \ accuracy:  72.480%
epoch:  1  batch:   10 [  6000/60000]  loss: 0.47113931 \ accuracy:  83.100%
epoch:  1  batch:   20 [ 12000/60000]  loss: 0.42378384 \ accuracy:  83.708%
epoch:  1  batch:   30 [ 18000/60000]  loss: 0.41309258 \ accuracy:  83.961%

#### Time: 100 Seconds / Accuracy: 90.35%

### CNN_Tanh_Sigmoid

In [13]:
class MultiClassPerceptron_Tanh(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1_T = nn.Conv2d(1, 6, 3, 1)
        self.conv2_T = nn.Conv2d(6, 16, 3, 1)
        self.fc1_T = nn.Linear(5*5*16, 120)
        self.fc2_T = nn.Linear(120, 84)
        self.fc3_T = nn.Linear(84, 10)
        
    def forward(self, X):
        X = torch.tanh(self.conv1_T(X))
        X = F.max_pool2d(X, 2, 2)
        X = torch.tanh(self.conv2_T(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1_T(X))
        X = F.relu(self.fc2_T(X))
        X = self.fc3_T(X)
        return F.sigmoid(X)

In [14]:
torch.manual_seed(22)
model_tanh = MultiClassPerceptron_Tanh()
model_tanh

MultiClassPerceptron_Tanh(
  (conv1_T): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2_T): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1_T): Linear(in_features=400, out_features=120, bias=True)
  (fc2_T): Linear(in_features=120, out_features=84, bias=True)
  (fc3_T): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
tanh_criterion = nn.CrossEntropyLoss()
tanh_optimizer = torch.optim.Adam(model_tanh.parameters(), lr=0.001)

In [16]:
import time
start_time = time.time()

epochs = 10
tanh_trn_losses = []
tanh_tst_losses = []
tanh_trn_correct = []
tanh_tst_correct =[]

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1
        
        y_pred = model_tanh(X_train)
        loss = tanh_criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        tanh_optimizer.zero_grad()
        loss.backward()
        tanh_optimizer.step()
        
        if b%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{600*b:6}/60000]  loss: {loss.item():10.8f} \ accuracy: {trn_corr.item()*100/(600*b):7.3f}%')
            
    tanh_trn_losses.append(loss.item())
    tanh_trn_correct.append(trn_corr.item())
    
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            
            y_val = model_tanh(X_test)
            
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()
            
    
    loss = tanh_criterion(y_val, y_test)
    tanh_tst_losses.append(loss)
    tanh_tst_correct.append(tst_corr)
    

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

/opt/anaconda3/envs/pytorchenv/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch:  0  batch:   10 [  6000/60000]  loss: 2.23697138 \ accuracy:  31.950%
epoch:  0  batch:   20 [ 12000/60000]  loss: 2.03798485 \ accuracy:  41.908%
epoch:  0  batch:   30 [ 18000/60000]  loss: 1.86863792 \ accuracy:  45.533%
epoch:  0  batch:   40 [ 24000/60000]  loss: 1.78770447 \ accuracy:  45.746%
epoch:  0  batch:   50 [ 30000/60000]  loss: 1.78253353 \ accuracy:  45.813%
epoch:  0  batch:   60 [ 36000/60000]  loss: 1.74913371 \ accuracy:  45.994%
epoch:  0  batch:   70 [ 42000/60000]  loss: 1.71817338 \ accuracy:  46.388%
epoch:  0  batch:   80 [ 48000/60000]  loss: 1.72578454 \ accuracy:  46.944%
epoch:  0  batch:   90 [ 54000/60000]  loss: 1.69708025 \ accuracy:  48.039%
epoch:  0  batch:  100 [ 60000/60000]  loss: 1.70152164 \ accuracy:  49.060%
epoch:  1  batch:   10 [  6000/60000]  loss: 1.68664443 \ accuracy:  60.017%
epoch:  1  batch:   20 [ 12000/60000]  loss: 1.68907177 \ accuracy:  59.508%
epoch:  1  batch:   30 [ 18000/60000]  loss: 1.67280805 \ accuracy:  60.056%

#### Time: 102 Seconds / Accuracy: 80.785%

### CNN_Tanh_SoftMax

In [8]:
class MultiClassPerceptron_Tanh_SM(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, X):
        X = torch.tanh(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = torch.tanh(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*5*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [9]:
torch.manual_seed(22)
model_tanh_SM = MultiClassPerceptron_Tanh_SM()
model_tanh_SM

MultiClassPerceptron_Tanh_SM(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [11]:
tanh_SM_criterion = nn.CrossEntropyLoss()
tanh_SM_optimizer = torch.optim.Adam(model_tanh_SM.parameters(), lr=0.001)

In [12]:
import time
start_time = time.time()

epochs = 10
tanh_trn_losses = []
tanh_tst_losses = []
tanh_trn_correct = []
tanh_tst_correct =[]

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train, y_train) in enumerate(train_loader):
        b+=1
        
        y_pred = model_tanh_SM(X_train)
        loss = tanh_SM_criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        tanh_SM_optimizer.zero_grad()
        loss.backward()
        tanh_SM_optimizer.step()
        
        if b%10 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{600*b:6}/60000]  loss: {loss.item():10.8f} \ accuracy: {trn_corr.item()*100/(600*b):7.3f}%')
            
    tanh_trn_losses.append(loss.item())
    tanh_trn_correct.append(trn_corr.item())
    
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            
            y_val = model_tanh_SM(X_test)
            
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()
            
    
    loss = tanh_SM_criterion(y_val, y_test)
    tanh_tst_losses.append(loss)
    tanh_tst_correct.append(tst_corr)
    

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

epoch:  0  batch:   10 [  6000/60000]  loss: 2.05739188 \ accuracy:  34.733%
epoch:  0  batch:   20 [ 12000/60000]  loss: 1.44330263 \ accuracy:  45.875%
epoch:  0  batch:   30 [ 18000/60000]  loss: 1.04534376 \ accuracy:  51.239%
epoch:  0  batch:   40 [ 24000/60000]  loss: 0.79805136 \ accuracy:  54.917%
epoch:  0  batch:   50 [ 30000/60000]  loss: 0.83621693 \ accuracy:  57.633%
epoch:  0  batch:   60 [ 36000/60000]  loss: 0.75412875 \ accuracy:  59.853%
epoch:  0  batch:   70 [ 42000/60000]  loss: 0.62224966 \ accuracy:  61.795%
epoch:  0  batch:   80 [ 48000/60000]  loss: 0.65670025 \ accuracy:  63.144%
epoch:  0  batch:   90 [ 54000/60000]  loss: 0.60951126 \ accuracy:  64.572%
epoch:  0  batch:  100 [ 60000/60000]  loss: 0.64463162 \ accuracy:  65.672%
epoch:  1  batch:   10 [  6000/60000]  loss: 0.60726339 \ accuracy:  76.067%
epoch:  1  batch:   20 [ 12000/60000]  loss: 0.63884711 \ accuracy:  76.408%
epoch:  1  batch:   30 [ 18000/60000]  loss: 0.57683945 \ accuracy:  76.561%

#### Time: 68 Seconds / Accuracy: 87.56%